In [2]:
from fastapi import FastAPI

import pandas as pd
import uvicorn

In [3]:
app = FastAPI()
genreFunction = pd.read_parquet('./data/genre_functions.parquet')

In [248]:
items = pd.read_parquet('./data/items.parquet')
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5094077,76561198329548331,346330,BrainBread 2,0,0
5094078,76561198329548331,373330,All Is Dust,0,0
5094079,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5094080,76561198329548331,521570,You Have 10 Seconds 2,4,4


In [120]:

# We apply the str.lower() method so that it does not matter how the word is written.
genreFunction['genres'] = genreFunction['genres'].str.lower()
genreFunction['genres'].unique()

array(['action', 'indie', 'racing', 'strategy', 'rpg', 'simulation',
       'casual', 'adventure', 'sports', 'free to play',
       'massively multiplayer', 'early access',
       'animation &amp; modeling', 'education', 'software training',
       'utilities', 'web publishing', 'design &amp; illustration',
       'audio production', 'video production', 'photo editing'],
      dtype=object)

In [212]:
genreFunction

,genres,year,user_id,playtime_forever
0,action,1997,UTNerd24,5
1,action,1997,I_DID_911_JUST_SAYING,0
2,action,1997,76561197962104795,0
3,action,1997,r3ap3r78,0
4,action,1997,saint556,13
...,...,...,...,...
9761456,action,2004,76561198273508956,0
9761457,action,2004,76561198282090798,0
9761458,action,2004,943525,0
9761459,action,2004,76561198283312749,9


In [211]:
mostPlayed = genreFunction.groupby('user_id')['playtime_forever'].sum()
mostPlayed.head()

user_id
--000--              228532
--ace--               91837
--ionex--             47185
-2SV-vuLB-Kg          77824
-404PageNotFound-    443507
Name: playtime_forever, dtype: int64

In [106]:
# The app.get decorator is used to associate the function with an HTTP GET request.
# '/PlayTimeGenre/{genre}/' is the path for the endpoint, the {genre} is the parameter.
@app.get('/PlayTimeGenre/{genre}/')
def PlayTimeGenre( genre : str ):
    '''
    Find the genre with the most playtime hours.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    int
        Year when the highest number of hours played was recorded for that genre.
    
    Examples
    -------
    >>> PlayTimeGenre('Action')
    2012
    >>> PlayTimeGenre('Indie')
    2006

    '''
    # We look in our DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    # We calculate the total playtime for each year in the dataset.
    results = genres.groupby('year')['playtime_forever'].sum()
    
    # Locate the index where the maximum value is. In this case the most played genre.
    idResut = results.idxmax()

    
    
    if genres.empty:
        return f'The genre {genre} does not exist.'

    
    return f'Year with the most playtime hours for {genre}: {idResut}'

In [107]:
test = PlayTimeGenre('RPG')
print(test)

Year with the most playtime hours for RPG: 2011


# Segunda Función

In [244]:
@app.get('/PlayTimeGenre/{genre}/')
def UserForGenre( genre : str ):
    '''
    Find the user with the most hours played by genre, as well as hours played for each year.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    list
        The user name and the amount of hours played by year for that particular genre.
    
    Examples
    -------
    >>> UserForGenre('Simulation')
    UserName is the user with the most playtime for the genre "Action" with 23721 hours played.

    Year            2003    2006    2009    2010    2011    2012    2013    2014    2015    2016    
    Hours Played      0      0      2037    4102    1968     223     323    342     1224     112   
    '''
    
    
    # Look in the DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    if genres.empty:
        return f'The genre {genre} does not exist.'

    # Group the DataSet by user ID, then sum the amount of hours played.
    mostPlayed = genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Locate the index for the player.
    player = genres.loc[genres['playtime_forever'].idxmax()]['user_id']

    # Filter the DataSet with only the player id
    filteredDFWithPlayerID = (genres[genres['user_id'] == player])

    # Create a new DF with just the year and playtime_forever columns
    hoursPlayedByYear = filteredDFWithPlayerID.groupby('year')['playtime_forever'].sum()
    
    
    print(f'{player} is the user with the most playtime for the genre "{genre.capitalize()}" with {filteredDFWithPlayerID["playtime_forever"].sum()} hours played.')
    print(f'\nYear\tHours Played\n')
    for year, hour in hoursPlayedByYear.items():
        print(f'{year}\t{hour}\n')
    


In [ ]:
UserForGenre('actiON')

# 3rd Function

In [ ]:

def UsersRecommend( year : int ):
    '''
    
    
    '''
    pass